# Accessing satellite data from AWS with Python

In [ ]:
!pip uninstall folium -y

!pip install sentinelhub
!pip install geopandas
!pip install shapely
!pip install sentinelsat


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 3.7 MB/s 


This example notebook shows how to obtain Sentinel-2 imagery and additional data from [AWS S3 storage buckets](https://aws.amazon.com/s3/). The data at AWS is the same as original S-2 data provided by ESA.

The ```sentinelhub``` package supports obtaining data by specifying products or by specifying tiles. It can download data either to the same file structure as it is at AWS or it can download data into original ```.SAFE``` file structure [introduced by ESA](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/data-formats).

Before testing any of the examples below please check [Configuration paragraph](http://sentinelhub-py.readthedocs.io/en/latest/configure.html#amazon-s3-capabilities) for details about configuring AWS credentials and information about charges.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sentinelhub import WebFeatureService, BBox, CRS, DataSource, SHConfig
import geopandas as gpd
from shapely.geometry import box
import pandas as pd
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from collections import namedtuple

pd.set_option('display.max_rows', 5000)

In [ ]:
# Getting reference Bounding Box


In [65]:
minx = -48.44006632
xmax = -48.33143401
ymin = -1.46577103
ymax = -1.22102593

Bounding_Box = box(minx, ymin, xmax, ymax)

print('minx``, ``miny``, ``maxx``, ``maxy`')
print(Bounding_Box)

minx``, ``miny``, ``maxx``, ``maxy`
POLYGON ((-48.33143401 -1.46577103, -48.33143401 -1.22102593, -48.44006632 -1.22102593, -48.44006632 -1.46577103, -48.33143401 -1.46577103))


In [ ]:

def readUserKeys():
  with open("credentials.txt") as f:
    username, password = f.read().replace("\n", "").split(":")
  return username, password

username, password = readUserKeys()

api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

# search by polygon, time, and SciHub query keywords
footprint = Bounding_Box.wkt
products = api.query(footprint,
                     date=(date(2008, 1, 1), date(2017, 12, 29)),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))

print('N° of images queried: ', len(products))


N° of images queried:  24


In [ ]:

# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

products_df.head()


,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,beginposition,endposition,ingestiondate,...,platformname,size,tileid,hv_order_tileid,filename,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier
7bc9678e-8a97-46a4-868e-a04dd0e4bd36,S2A_MSIL1C_20160416T134042_N0201_R124_T22MGD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2016-04-16T13:40:42.03Z, Instrument: MSI...",false,2016-04-16 13:40:42.030,2016-04-16 13:40:42.030,2016-04-16 13:40:42.030,2018-11-19 18:26:54.455,...,Sentinel-2,772.10 MB,22MGD,MD22G,S2A_MSIL1C_20160416T134042_N0201_R124_T22MGD_2...,S2A_MSIL1C_20160416T134042_N0201_R124_T22MGD_2...,7bc9678e-8a97-46a4-868e-a04dd0e4bd36,S2A_OPER_MSI_L1C_TL_SGS__20160416T185453_A0042...,S2A_OPER_MSI_L1C_TL_SGS__20160416T185453_A0042...,S2A_OPER_MSI_L1C_DS_SGS__20160416T185453_S2016...
233c8a5f-8f53-4c72-a4b4-3ed797f14328,S2A_MSIL1C_20160506T134212_N0202_R124_T22MGD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2016-05-06T13:42:12.027Z, Instrument: MS...",false,2016-05-06 13:42:12.027,2016-05-06 13:42:12.027,2016-05-06 13:42:12.027,2018-11-09 13:54:58.918,...,Sentinel-2,771.04 MB,22MGD,MD22G,S2A_MSIL1C_20160506T134212_N0202_R124_T22MGD_2...,S2A_MSIL1C_20160506T134212_N0202_R124_T22MGD_2...,233c8a5f-8f53-4c72-a4b4-3ed797f14328,S2A_OPER_MSI_L1C_TL_MTI__20160506T200542_A0045...,S2A_OPER_MSI_L1C_TL_MTI__20160506T200542_A0045...,S2A_OPER_MSI_L1C_DS_MTI__20160506T200542_S2016...
c9cb6857-f020-438b-a2e4-45be0e1529d4,S2A_MSIL1C_20160705T134212_N0204_R124_T22MGD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2016-07-05T13:42:12.026Z, Instrument: MS...",false,2016-07-05 13:42:12.026,2016-07-05 13:42:12.026,2016-07-05 13:42:12.026,2018-10-17 14:42:48.761,...,Sentinel-2,760.04 MB,22MGD,MD22G,S2A_MSIL1C_20160705T134212_N0204_R124_T22MGD_2...,S2A_MSIL1C_20160705T134212_N0204_R124_T22MGD_2...,c9cb6857-f020-438b-a2e4-45be0e1529d4,S2A_OPER_MSI_L1C_TL_SGS__20160705T151548_A0054...,S2A_OPER_MSI_L1C_TL_SGS__20160705T151548_A0054...,S2A_OPER_MSI_L1C_DS_SGS__20160705T151548_S2016...
174ab880-5f71-45a9-9e00-46d586ba7484,S2A_MSIL1C_20160725T134212_N0204_R124_T22MGD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2016-07-25T13:42:12.026Z, Instrument: MS...",false,2016-07-25 13:42:12.026,2016-07-25 13:42:12.026,2016-07-25 13:42:12.026,2018-09-12 17:03:57.993,...,Sentinel-2,720.12 MB,22MGD,MD22G,S2A_MSIL1C_20160725T134212_N0204_R124_T22MGD_2...,S2A_MSIL1C_20160725T134212_N0204_R124_T22MGD_2...,174ab880-5f71-45a9-9e00-46d586ba7484,S2A_OPER_MSI_L1C_TL_SGS__20160725T184136_A0056...,S2A_OPER_MSI_L1C_TL_SGS__20160725T184136_A0056...,S2A_OPER_MSI_L1C_DS_SGS__20160725T184136_S2016...
d9d35683-8e4b-4dbc-ba4f-43ef6994f7fd,S2B_MSIL1C_20170715T134209_N0205_R124_T22MGD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-07-15T13:42:09.027Z, Instrument: MS...",false,2017-07-15 13:42:09.027,2017-07-15 13:42:09.027,2017-07-15 13:42:09.027,2018-09-01 03:12:41.179,...,Sentinel-2,746.58 MB,22MGD,MD22G,S2B_MSIL1C_20170715T134209_N0205_R124_T22MGD_2...,S2B_MSIL1C_20170715T134209_N0205_R124_T22MGD_2...,d9d35683-8e4b-4dbc-ba4f-43ef6994f7fd,S2B_OPER_MSI_L1C_TL_MTI__20170715T200433_A0018...,S2B_OPER_MSI_L1C_TL_MTI__20170715T200433_A0018...,S2B_OPER_MSI_L1C_DS_MTI__20170715T200433_S2017...


In [ ]:
# Sorting by cloudcover and data acquisition date
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'datatakesensingstart'], ascending=[True, True])
products_df_sorted.shape

(24, 36)

In [ ]:
import os, glob

def check_wild_file_existance(directory, basename):

    filenames = glob.glob(os.path.join(directory, basename + '*'))
    
    if len(filenames):
        return True
    else:
        return False

In [ ]:
minx, miny, maxx, maxy = np.round(Bounding_Box.bounds, 0).astype(int)

directory_path = r'IMAGENS_SENTINEL\S2\downloads\BB_minx{0}_miny{1}_maxx{2}_maxy{3}'.format(minx, miny, maxx, maxy)

if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [ ]:

for index in products_df_sorted.index:                                              
    # download sorted and reduced products
    
    product_info = api.get_product_odata(index)

    # making sure that the data is not yet downloaded locally:
    
    if not check_wild_file_existance(directory_path, product_info['title']):

        if product_info['Online']:
            print('Download start of \n\t {0}'.format(product_info['title']), end='\n'*2)

            # api.download(index, directory_path=directory_path)

        else:
            print('{} is off-online. Skipping.'.format(product_info['title']), end='\n'*2)
            
    else:
        print('{0} is already in folder.'.format(product_info['title']), '\n\t Skipping download', end='\n'*2)


S2A_MSIL1C_20170720T134211_N0205_R124_T22MGD_20170720T134211 is off-online. Skipping.

S2A_MSIL1C_20170829T134211_N0205_R124_T22MGD_20170829T134210 is off-online. Skipping.

S2A_MSIL1C_20170809T134211_N0205_R124_T22MGD_20170809T134212 is off-online. Skipping.

S2B_MSIL1C_20170715T134209_N0205_R124_T22MGD_20170715T134210 is off-online. Skipping.

S2A_MSIL1C_20161102T134212_N0204_R124_T22MGD_20161102T134208 is off-online. Skipping.

S2B_MSIL1C_20170804T134209_N0205_R124_T22MGD_20170804T134209 is off-online. Skipping.

S2A_MSIL1C_20160506T134212_N0202_R124_T22MGD_20160506T134237 is off-online. Skipping.

S2B_MSIL1C_20171003T134159_N0205_R124_T22MGD_20171003T134155 is off-online. Skipping.

S2A_MSIL1C_20170918T134211_N0205_R124_T22MGD_20170918T134207 is off-online. Skipping.

S2B_MSIL1C_20171222T134159_N0206_R124_T22MGD_20171222T151240 is off-online. Skipping.

S2A_MSIL1C_20160814T134212_N0204_R124_T22MGD_20160814T134213 is off-online. Skipping.

S2B_MSIL1C_20171202T134159_N0206_R124_T22MG

KeyboardInterrupt: ignored